# Linear Regression using La Classy

In [1]:
import {
  OLSSolver
} from "jsr:@lala/classy@1.0.1";

Let's initialize the predictor and target variables.

We will make our model learn the function `y1 = 9x1 + 2x2 + 5x3 + 8`, `y2 = 3x1 + 4x2 + 3x3 + 5`.

In [2]:
const X = [[5, 6, 10], [9, 13, 2], [12, 45, 92], [8, 2, 13], [43, 4, 19]]
const y = X.map(x => ([x[0] * 9 + x[1] * 2 + x[2] * 5 + 8, x[0] * 3 + x[1] * 4 + x[2] * 3 + 5]))

y

[ [ 115, 74 ], [ 125, 90 ], [ 666, 497 ], [ 149, 76 ], [ 498, 207 ] ]

In [3]:
const X_train = { data: Float64Array.from(X.flat()), shape: [5, 3] };
const y_train = { data: Float64Array.from(y.flat()), shape: [5, 2] };


Now that we have prepared our inputs, we can initialize our solver. Since we are performing linear regression, let's try out an Ordinary Least Squares solver.

In [4]:
const solver = new OLSSolver();

We can then train our model using the data we acquired.

In [5]:
const time = performance.now();
solver.train(X_train, y_train, {
  fit_intercept: true,
});
console.log(`training time: ${performance.now() - time}ms`);

training time: 0.3636999999998807ms


The model is trained, now it is time to evaluate its performance on our testing dataset

In [6]:
const res = solver.predict({data: Float64Array.from([10, 13, 8]), shape: [1, 3]})
res.shape

[ 1, 2 ]

In [7]:
[Array.from(res.data), [10 * 9 + 13 * 2 + 8 * 5 + 8, 10 * 3 + 13 * 4 + 8 * 3 + 5]]

[ [ 164.00000000000037, 111.00000000000037 ], [ 164, 111 ] ]